## Inference from saved models

In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import utils as ut
import siamese_models as sm

2025-01-14 08:26:15.586412: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-14 08:26:15.597615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736839575.611516 2525086 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736839575.615737 2525086 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 08:26:15.629900: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Models
Load a single model

In [2]:
resnet_path = './models/ResNet_s5.pt'

# load model architecture
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_s5 = sm.SiameseNetworkResnet().to(device)  # Use the correct model class
# load model through saved state_dict
resnet_s5.load_state_dict(torch.load(resnet_path))
resnet_s5.eval()

/tmp/ipykernel_2525086/3086697005.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet_s5.load_state_dict(torch.load(resnet_path))


SiameseNetworkResnet(
  (resnet): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

Load all models 

In [18]:
# load model architecture
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_names = {'ResNet': sm.SiameseNetworkResnet().to(device), 'MobileNet': sm.SiameseNetworkMobnet().to(device), 'ResNeXt': sm.SiameseNetworkNext().to(device),
              'VGGNet': sm.SiameseNetworkVGGnet().to(device), 'EffNet': sm.SiameseNetworkEffnet().to(device) }
model_paths = [f"./models/{model}_s5.pt" for model in model_names.keys()]

# load models through saved state_dict
for model, model_path in zip(model_names.values(), model_paths):
    model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_2525086/1240181476.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Data

In [4]:
# Assume ds_folds[0]['test'] is your test dataset
ds_folds = ut.siamese_noise_dataset_fold_range(test_size=0.1, noise_size=5, n_splits=2)
test_loader = DataLoader(ds_folds[0]['test'].with_format("torch"), batch_size=32, shuffle=False)

Total dataset size: 3150
Number of T1 images: 1476
Number of T2 images: 1674


### Evaluation

In [10]:
def evaluation(model):
    all_predictions, all_targets = [], []

    with torch.no_grad():
        for i in test_loader:
            images_1 = i['slice1'].to(device, dtype=torch.float32)
            images_2 = i['slice2'].to(device, dtype=torch.float32)
            targets = i['label'].to(device, dtype=torch.float32)

            outputs = model(images_1, images_2).squeeze()
            # Threshold at 0.5 to get binary predictions
            predictions = (outputs > 0.5).float()

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())


    # Evaluate metrics
    all_targets = np.array(all_targets)
    all_predictions = np.array(all_predictions)

    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions)
    recall = recall_score(all_targets, all_predictions)
    f1 = f1_score(all_targets, all_predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")


In [13]:
model = resnet_s5.eval()
evaluation(model)

Accuracy: 0.7753
Precision: 0.7771
Recall: 0.8095
F1 Score: 0.7930


In [28]:
for model_type, model in zip(model_names.keys(), model_names.values()):
    print(f"{model_type} : ")
    evaluation(model.eval())
    print(".")

ResNet : 
Accuracy: 0.7753
Precision: 0.7771
Recall: 0.8095
F1 Score: 0.7930
.
MobileNet : 
Accuracy: 0.6424
Precision: 0.6471
Recall: 0.7202
F1 Score: 0.6817
.
ResNeXt : 
Accuracy: 0.5316
Precision: 0.5649
Recall: 0.5179
F1 Score: 0.5404
.
VGGNet : 
Accuracy: 0.7785
Precision: 0.7475
Recall: 0.8810
F1 Score: 0.8087
.
EffNet : 
Accuracy: 0.4968
Precision: 0.6364
Recall: 0.1250
F1 Score: 0.2090
.
